# Gaussian Mixture Model

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from scipy.stats import multivariate_normal as gauss

In [2]:
# reading datapoints from csv file
data = pd.read_csv('weight-height.csv',header = None, skiprows = 1)
data = np.array(data)
data = data[0:1000, :]
N,d = data.shape

In [3]:
#finding initial mean(centroids),covariance,prior probabilities(pie) using K-means

n_clusters = int(input("Enter no. of clusters : "))
thres = float(input("Enter threshold : "))
pie = np.zeros((n_clusters))
kmeans = KMeans(n_clusters, random_state=0).fit(data)

# Prior probabilities(pie)
for i in range(n_clusters):
    z = np.argwhere(kmeans.labels_==i)
    pie[i] = len(z)/N

# Means
mean = kmeans.cluster_centers_
print(mean[0])
# covariance matrix
covs = {}
for i in range(n_clusters):
    covs[i] =np.identity(d)
print(mean)

Enter no. of clusters : 2
Enter threshold : 0.01
[ 70.83417069 202.26995774]
[[ 70.83417069 202.26995774]
 [ 66.95339717 170.63591974]]


In [4]:
#Finding Posterior Probabilities
cov_k = np.zeros((n_clusters,d,d))
l,l_new = 0,0
gaussian = []
gam = []
pie_k = []
mu_k = []


while True:
    for i in range(n_clusters):
        g = np.random.multivariate_normal(mean[i], covs[i],N).T
        gaussian.append(g)
    gaussian = np.array(gaussian)
    
    for i in range(n_clusters):
        n = pie[i]* gauss.pdf(np.array(data),mean[i],covs[i])
        dn = n/np.sum(n)
        gam.append(dn)
        
        
    # new prior Probabilities
   
    for i in range(n_clusters):
        pie_k.append(np.sum(gam[i],axis = 0)/N)
    pie_k = np.array(pie_k)

    gamma = np.array(gam).T

    # new mean
    for i in range(n_clusters):
        s = np.sum(gamma.T @ data,axis = 1)/np.sum(gam[i],axis = 0)
        mu_k.append(s)

    mu_k = np.array(mu_k)


    # new covariance
    for i in range(n_clusters):
        for j in range(N): 
            cov_k[i,:,:] += (gamma[j,i]*np.reshape(data[j,:]-mu_k[i], (d,1))*(data[j,:]-mu_k[i]))/np.sum(gam[i],axis = 0)


    # log likelihood
    for i in range(n_clusters):
        l += np.log( np.sum(pie_k[i]* gauss.pdf(np.array(data),mu_k[i],cov_k[i])))
       
    if np.abs(l_new - l) <= thres:    
        print("final mean : \n",mu_k)
        print("Final mixing coefficient : ",pie_k)
        print("final covariance : ",cov_k)
        break
    print('error',np.abs(l_new - l))

    pie_k = pie_k.tolist()
    mu_k = mu_k.tolist()
    mean = []
    mean = mu_k
    pie = []
    covs = cov_k

    pie = pie_k
    l_new = l
    l = 0

    gaussian = []
    gam = []
    pie_k = []
    mu_k = []

error 21.663866967178283
error 0.2406132070033884
error 0.28114896803847245
error 0.22094671039223002
error 0.17293699180573796
error 0.17164984249732385
error 0.1869575243626045
error 0.20647622787586073
error 0.25509559779524693
error 0.3225454774640468
error 0.3595385044940116
error 0.3428723584895934
error 0.29350963935903707
error 0.24716607619523145
error 0.2136141144053667
error 0.1888076280141071
error 0.16922538257683328
error 0.15322348341746306
error 0.13993369532894917
error 0.128767297152514
error 0.11928185056765983
error 0.111138895021476
error 0.10407894136239548
error 0.09790212431126832
error 0.09245325660769055
error 0.08761063901363286
error 0.08327781876865714
error 0.07937754202166758
error 0.07584730043816279
error 0.07263602784363599
error 0.06970162767181876
error 0.06700910411483108
error 0.06452913550138106
error 0.06223697458941757
error 0.06011159282731171
error 0.05813500843152042
error 0.056291754253933846
error 0.05456845291811874
error 0.052953474990324